# This Notebook Simulates and Animates Milling About

## Required Imports

In [94]:
%load_ext autoreload
%autoreload 2

import math
import numpy as np

from fish import Fish

from variables import Variables
from camera import Camera

from interaction import Interaction
from environment import Environment
from channel import Channel
from observer import Observer
from dynamics import Dynamics

from utils import generate_distortion, generate_fish, run_simulation, run_simulation_with_coverage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simulation of Milling About

Change parameters here, e.g. number of fish or initializations.

In [95]:
from events import Homing

run_time = 90 # in seconds
num_fish = 5
arena_size = np.array([1780, 1780, 1170])
res = 10
camera_depth = 100
arena_center = arena_size / 2.0
initial_spread = 2000
fish_pos = initial_spread * np.random.rand(num_fish, 3) + arena_center - initial_spread / 2.0
fish_vel = np.zeros((num_fish, 3))
fish_phi = math.pi * np.random.rand(num_fish, 1)
fish_vphi = np.zeros((num_fish, 1))
clock_freqs = 1
verbose = False

distortion = generate_distortion(type='none', magnitude=130, n=math.ceil(arena_size[0]/10)+1, show=False)
environment = Environment(
    arena_size=arena_size,
    node_pos=fish_pos,
    node_vel=fish_vel,
    node_phi=fish_phi,
    node_vphi=fish_vphi,
    distortion=distortion,
    prob_type='binary',
    conn_thres=3000,
    conn_drop=1,
    noise_magnitude=10,
    verbose=verbose
)
interaction = Interaction(environment, verbose=verbose)
channel = Channel(environment)
dynamics = Dynamics(environment, clock_freq=clock_freqs)
variables = Variables()
camera = Camera(camera_depth)

fish = generate_fish(
    n=num_fish, 
    channel=channel, 
    interaction=interaction,
    dynamics=dynamics,
    variables=variables,
    camera=camera,
    res=res,
    w_blindspot=50,
    r_blocking=65,
    target_dist=130*1.75,
    lim_neighbors=[2,3],
    neighbor_weights=1.0,
    fish_max_speeds=130,
    clock_freqs=clock_freqs,
    verbose=verbose
)
channel.set_nodes(fish)

observer = Observer(fish=fish, environment=environment, channel=channel)

missing_aircraft = Homing()
for i in range(1, run_time): 
    observer.instruct(event=missing_aircraft, rel_clock=i, pos=arena_center, fish_all=True)
    
run_simulation_with_coverage(fish=fish, observer=observer, run_time=run_time, dark=False, white_axis=False, no_legend=True)



400
Please wait patiently 90 seconds. Thanks.


/Users/lucas/OneDrive - Harvard University/Senior/Fall/CS289/BlueSim-master/camera.py:35: RuntimeWarning: invalid value encountered in true_divide
  x_c_left = x_c / np.linalg.norm(x_c)
/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1804: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


slots [ 39.  35.  44.  27.  25.  28.  28.  34.  31.  28.  40.  34.  50.  29.  25.
  28.  29.  34.  31.  26.  42.  34.  47.  29.  25.  28.  29.  36.  31.  27.
  40.  34.  48.  29.  25.  27.  28.  35.  29.  28.  39.  35.  46.  28.  23.
  25.  25.  35.  29.  28.  38.  35.  46.  28.  23.  24.  23.  33.  30.  28.
  39.  33.  48.  27.  22.  23.  23.  33.  30.  28.  40.  34.  49.  27.  22.
  23.  22.  31.  30.  28.  38.  32.  49.  27.  22.  23.  21.  29.  28.  28.
  38.  32.  50.  27.  22.  23.  21.  28.  27.  28.  20.  29.  27.  32.  39.
  42.  38.  49.  42.  31.  20.  29.  27.  33.  39.  42.  38.  49.  42.  31.
  20.  29.  27.  33.  39.  42.  38.  49.  42.  31.  20.  29.  27.  33.  39.
  42.  38.  51.  42.  31.  20.  29.  27.  33.  39.  42.  38.  51.  42.  31.
  20.  29.  27.  33.  39.  42.  38.  51.  42.  31.  20.  30.  27.  33.  39.
  42.  38.  51.  42.  31.  20.  30.  27.  33.  39.  42.  38.  51.  42.  31.
  20.  30.  27.  33.  39.  42.  38.  51.  42.  31.  20.  30.  27.  33.  39.
  42. 

## Prepare Data for Animation

In [56]:
# Get fish data from observer
data = np.zeros((6, run_time, num_fish+2))
data[0, :, :-2] = np.transpose(np.array(observer.x))
data[1, :, :-2] = np.transpose(np.array(observer.y))
data[2, :, :-2] = np.transpose(np.array(observer.z))
data[3, :, :-2] = np.transpose(np.array(observer.vx))
data[4, :, :-2] = np.transpose(np.array(observer.vy))
data[5, :, :-2] = np.transpose(np.array(observer.vz))

# Add center
data[0, :, -2] = 1780/2
data[1, :, -2] = 1780/2
data[2, :, -2] = 1170/2+45
data[3, :, -2] = 0
data[4, :, -2] = 0
data[5, :, -2] = -1
data[0, :, -1] = 1780/2
data[1, :, -1] = 1780/2
data[2, :, -1] = 1170/2-45
data[3, :, -1] = 0
data[4, :, -1] = 0
data[5, :, -1] = 1

## Save Data

In [66]:
np.save('millingabout', data)

## Animate Simulated Data

Change the speed-up factor of the animation in the variable `speed`.

In [ ]:
# Imports
import ipyvolume as ipv
from ipyvolume.moviemaker import MovieMaker
import ipywidgets as widgets
import ipywebrtc as webrtc
import matplotlib.cm as cm
import time

# Data handling
x, y, z, vx, vy, vz = data
speed = 10 # speed up animation 10 times

# Colors
v = np.sqrt(x**2 + y**2 + z**2)
v -= v.min(); v /= v.max();
colors = np.array([cm.Blues(k) for k in v])
colors[:, -2:, :] = cm.Reds(0.5) # center is red

# Figure
fig = ipv.figure()
ipv.xlim(0, 1780)
ipv.ylim(0, 1780)
ipv.zlim(0, 1170)
ipv.style.use('dark')
quiver = ipv.quiver(x, y, z, vx, vy, vz, size=10, color=colors[:,:,:3])
ipv.animation_control(quiver, interval=1000/speed)
ipv.show()

## Save Animation as html

In [ ]:
ipv.save('millingabout.html')
!open 'millingabout.html'